In [ ]:
import os

DOCUMENTS_PATH = os.getcwd() + r"/data"

In [ ]:
from langchain.document_loaders import DirectoryLoader

loader = DirectoryLoader(DOCUMENTS_PATH)
documents = loader.load()

for document in documents:
    document.metadata['filename'] = document.metadata['source']
    document

Current tests to generate synthetic data to evaluate the RAG pipeline. The synthetic data (in this case) consists of a question and an answer.
The generation works like this: all documents (in this case all custom functions for DevExpress formulas) are iterated and the desired amount of test data is created. It randomly uses the documents (or multiple documents if needed) to create the question and answer.

In [ ]:
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context
from custom_evolutions import formula_evolution

try:
    import sys
    sys.path.append('/workspace')
    from internal_shared.models.ai import azure_model, critic_llm, azure_embeddings
except ImportError:
    print("No shared package found, using local package")


generator = TestsetGenerator.from_langchain(
    azure_model,
    critic_llm,
    azure_embeddings
)

# generate testset
testset = generator.generate_with_langchain_docs(documents, test_size=10, distributions={formula_evolution: 1.0}, with_debugging_logs=True)

In [ ]:
testset.to_pandas()

In [ ]:
import datetime

df = testset.to_pandas()

# get current timestamp
timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
# export as csv
df.to_csv(f"testset_{timestamp}.csv", sep=";", index=False)